In [1]:
import numpy as np
import random
from IPython.display import clear_output
import time

## HUNDIR LA FLOTA

In [5]:
# Importa colores para los prints
from colorama import init, Fore, Back, Style
import numpy as np
# Funcion limpiar consola
from IPython.display import clear_output
import time

class HundirLaFlota:
    def __init__(self, filas=10, columnas=10):
        """
        Inicializa el juego de hundir la flota.
        """
        self.filas = filas
        self.columnas = columnas
        self.tablero_jugador = np.full((self.filas, self.columnas), " ")
        self.tablero_ordenador = np.full((self.filas, self.columnas), " ")
        self.barcos = [5, 4, 3, 2] # Tamaño barcos
        self.letras_barcos = ['A', 'C', 'B', 'Y']  # Asignar letras a los barcos según su tamaño
        self.posiciones_barcos_jg = []
        self.posiciones_barcos_ordenador = []
        self.barcos_hundidos_jg = []
        self.barcos_hundidos_ordenador = []

    def colocar_barcos_aleatorios(self, tablero, jugador=False):
        """
        Coloca los barcos aleatoriamente en el tablero. Se utiliza una adyacencia entre barcos de forma que tiene que haber un hueco entre ellos.
        Cada barco está definido con una letra.
        """
        # Recorre las 2 listas de barcos y sus tamaños.
        for size, letra in zip(self.barcos, self.letras_barcos):
            # Si la letra ya está en el tablero, pasa a la siguiente asi hasta que finalice.
            if letra in tablero:
                continue
            # Esta variable se utiliza para diferenciar si esta en el tablero o no.
            colocado = False
            # Si colocado es False entra en el bucle.
            while not colocado:
                # Esto genera una posición aleatoria en el tablero sin que haya una adyacencia.
                fila = np.random.randint(0, self.filas)
                columna = np.random.randint(0, self.columnas)
                orientacion = np.random.choice(["horizontal", "vertical"])

                # Verifica si hay hueco en la posición aleatoria generada. si la hay pasa al siguiente if.
                if self.verificar_adyacencia(tablero, fila, columna, size):
                    # Verifica si el barco puede ser colocado sin salirse del tablero con la orientación y el tamaño
                    if orientacion == "horizontal" and columna + size <= self.columnas:
                        # Comprueba que no haya otros barcos en la posición deseada
                        if not np.any(tablero[fila, columna:columna + size] != " "):
                            # Posiciona el barco
                            tablero[fila, columna:columna + size] = letra
                            # Se indica que se ha colocado este barco
                            colocado = True
                    # Lo explicado pero en vertical
                    elif orientacion == "vertical" and fila + size <= self.filas:
                        if not np.any(tablero[fila:fila + size, columna] != " "):
                            tablero[fila:fila + size, columna] = letra
                            colocado = True

                    # Guarda las posiciones de los barcos colocados
                    if colocado:
                        # Condición que se le indica al método para saber si es el jugador o del ordenador
                        if jugador:
                            # i recorre todas las posiciones donde se coloca el barco
                            for i in range(size):
                                # Se guarda las posiciones en la lista.
                                if orientacion == "horizontal":
                                    self.posiciones_barcos_jg.append((fila, columna + i))
                                else:
                                    self.posiciones_barcos_jg.append((fila + i, columna))
                        else:
                            # Lo mismo que en lo anterior pero para la lista del ordenador
                            for i in range(size):
                                if orientacion == "horizontal":
                                    self.posiciones_barcos_ordenador.append((fila, columna + i))
                                else:
                                    self.posiciones_barcos_ordenador.append((fila + i, columna))

    def verificar_adyacencia(self, tablero, fila, columna, size):
        """
        Verifica si hay suficiente espacio alrededor de una posición para colocar un barco sin solaparse con otros barcos.
        """
        # i recorre todas las posiciones donde va ubicado el barco i revisa las posiciones en vertical
        # Se recorre desde -1 hasta size + 1, lo que permite verificar todas las posiciones alrededor del barco, incluyendo las posiciones donde estará el barco.
        for i in range(-1, size + 1):
            # j recorre todas las posiciones donde va ubicado el barco. j revisa las posiciones en horizontal
            for j in range(-1, size + 1):
                # Se verifica que este dentro del tablero el barco.
                if 0 <= fila + i < tablero.shape[0] and 0 <= columna + j < tablero.shape[1]:
                    # Verifica si esta el espacio vacio
                    if tablero[fila + i, columna + j] != " ":
                        # Si los huecos no esta vacios se devuelve False indicando que esta ocupado, si esta vacio True permitiendo que continue el flujo.
                        return False
        return True
    
    def disparo_simple(self, tablero, fila, columna):
        """
        Realiza un disparo en el tablero especificado.
        """
        # Comprueba que en la posición donde se ha indicado haya una letra que sea un barco.
        if tablero[fila, columna] in self.letras_barcos:
            # Se hace la asignación a la variable para que pueda verificar si el barco se ha hundido.
            barco_letra = tablero[fila, columna]
            # Si le da a una barco sustituye esa posición con "X"
            tablero[fila, columna] = "X"
            print(Fore.RED + f"¡Has tocado un barco {barco_letra}!")
            # Se llama a la funcion mostrar tableros que mantiene los tableros actualizados
            self.mostar_tableros()
            # Se llama a la funcion para que haga una verificación de si se ha hundido el barco
            self.verificar_hundimiento(tablero, barco_letra)  
            # Indica que el disparo ha sido exitoso
            return True  
        else:
            # Si no habia barco quedará reflejado en el tablero con un 0.
            tablero[fila, columna] = "0"
            print(Fore.BLUE+"¡Agua!")
            self.mostar_tableros()
            # Se indicará que el disparo fue al agua.
            return False  

    def turno_ordenador(self):
        """
        Realiza un turno del ordenador.
        """
        while True:
            fila = np.random.randint(0, self.tablero_ordenador.shape[0])
            columna = np.random.randint(0, self.tablero_ordenador.shape[1])
            if self.disparo_simple(self.tablero_jugador, fila, columna):
                # Si el disparo fue un acierto, el ordenador repite su turno
                continue
            else:
                break

    def verificar_hundimiento(self, tablero, barco_letra):
        """
        Verifica si se ha hundido un barco en el tablero especificado.
        """
        # si la letra donde se disparo no esta en la lista de barcos hundidos es que el barco no se ha hundido y compara el total 
        # de letras del barco que hay en el tablero, si es 0 indica que el barco esta totalmente hundido.
        if barco_letra not in self.barcos_hundidos_jg and np.sum(tablero == barco_letra) == 0:
            print(f"¡El barco {barco_letra} ha sido hundido!")
            # mete en una lista los barcos hundidos de cada tablero.
            if tablero is self.tablero_jugador:
                self.barcos_hundidos_jg.append(barco_letra)
            else:
                self.barcos_hundidos_ordenador.append(barco_letra)
    
    def mostrar_introduccion_y_normas(self):
        print(f"{Fore.CYAN}Bienvenido a Hundir la Flota{Style.RESET_ALL}")
        print("\nEste es un juego clásico de estrategia naval entre tú y el ordenador.")
        print("El objetivo es hundir todos los barcos del oponente antes de que hunda los tuyos.\n")

        print(f"{Fore.GREEN}Normas del juego:{Style.RESET_ALL}")
        print("1. Se colocarán de forma aleatoria los barcos en un tablero de 10x10.")
        print("2. Los barcos deben colocarse de forma que no se toquen, ni siquiera diagonalmente.")
        print("3. Cada jugador en su turno indica una coordenada para atacar en el tablero del oponente.")
        print("4. Recibirás un mensaje de '¡Has tocado un barco [Letra del Barco]!' si aciertas, o '¡Agua!' si fallas.")
        print("5. Cuando todas las partes de un barco han sido tocadas, se considera hundido. Se notificará con un mensaje específico.")
        print("6. El juego continua hasta que uno de los jugadores pierde todos sus barcos. Cuando no le quede ningun barco dispara al agua!, De este modo se celebra la victoria!\n")

        print(f"{Fore.YELLOW}Barcos disponibles:{Style.RESET_ALL}")
        print("- Portaaviones (A) de 5 casillas.")
        print("- Acorazado (C) de 4 casillas.")
        print("- Submarino (B) de 3 casillas.")
        print("- Patrullero (Y) de 2 casillas.\n")

        print("Para realizar un ataque, deberás introducir las coordenadas en forma de fila y columna (ej. 5 para la fila y 7 para la columna).")
        print(f"{Fore.RED}¡Prepárate, coloca tus barcos y que comience la batalla!{Style.RESET_ALL}\n")


    def jugar(self):
        """
        Inicia el juego empezando primero el jugador.
        """
        self.colocar_barcos_aleatorios(self.tablero_ordenador)
        self.colocar_barcos_aleatorios(self.tablero_jugador, jugador=True)
        self.mostar_tableros()
        
        
        while True:
            print("\n Tu turno")
            while True:
                # Indicar las posiciones que se quieren disparar.
                fila = int(input("Fila a disparar?"))
                columna = int(input("Columna a disparar?"))
                # Mientras acertemos el return será un True volviendo a ejecutar la funcion volviendo a empezar el bucle while.
                if self.disparo_simple(self.tablero_ordenador, fila, columna):
                    # Si el disparo fue exitoso, repetir el turno del jugador.
                    continue
                # Si fallamos nos delvolverá return False indicando que pase al break rompiendo el bucle.
                else:
                    break
            # Comprueba si quedan barcos en el tablero del ordenador para ver si hemos ganado.
            if not (np.any(self.tablero_ordenador == "A") or np.any(self.tablero_ordenador == "C") or np.any(self.tablero_ordenador == "B") or np.any(self.tablero_ordenador == "Y")):
                print("Has ganado!")
                break
            

            print("\n Turno del ordenador")
            # Entra en el turno del ordenador generando un disparo aleatorio en nuestro tablero.
            self.turno_ordenador()
            # Comprueba si ha ganado el ordenador
            if not (np.any(self.tablero_jugador == "A") or np.any(self.tablero_jugador == "C") or np.any(self.tablero_jugador == "B") or np.any(self.tablero_jugador == "Y")):
                print("Has perdido, no te quedan más barcos")
                break

    def mostar_tableros(self):
        clear_output(wait=True)
        time.sleep(2)
        print(f"Tablero ordenador \n {self.tablero_ordenador}")
        print(f"\nTablero jugador \n {self.tablero_jugador}")
        print(f"Estas son las posiciones donde están los barcos ordenador {self.posiciones_barcos_ordenador}")
        print(f"Estas son las posiciones donde están los barcos jugador {self.posiciones_barcos_jg}")
        time.sleep(2)



In [8]:
jugador2 = HundirLaFlota()
jugador2.mostrar_introduccion_y_normas()

Bienvenido a Hundir la Flota

Este es un juego clásico de estrategia naval entre tú y el ordenador.
El objetivo es hundir todos los barcos del oponente antes de que hunda los tuyos.

Normas del juego:
1. Se colocarán de forma aleatoria los barcos en un tablero de 10x10.
2. Los barcos deben colocarse de forma que no se toquen, ni siquiera diagonalmente.
3. Cada jugador en su turno indica una coordenada para atacar en el tablero del oponente.
4. Recibirás un mensaje de '¡Has tocado un barco [Letra del Barco]!' si aciertas, o '¡Agua!' si fallas.
5. Cuando todas las partes de un barco han sido tocadas, se considera hundido. Se notificará con un mensaje específico.
6. El juego continua hasta que uno de los jugadores pierde todos sus barcos. Cuando no le quede ningun barco dispara al agua!, De este modo se celebra la victoria!

Barcos disponibles:
- Portaaviones (A) de 5 casillas.
- Acorazado (C) de 4 casillas.
- Submarino (B) de 3 casillas.
- Patrullero (Y) de 2 casillas.

Para realizar un 

In [9]:
jugador2.jugar()

Tablero ordenador 
 [['B' 'B' 'B' ' ' ' ' ' ' '0' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'X']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'X']
 [' ' ' ' ' ' ' ' ' ' ' ' 'Y' ' ' ' ' 'X']
 [' ' ' ' ' ' ' ' ' ' ' ' 'Y' ' ' ' ' 'X']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'X']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'C' 'C' 'C' 'C']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']]

Tablero jugador 
 [[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'A']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'A']
 [' ' ' ' ' ' 'C' ' ' ' ' ' ' ' ' ' ' 'A']
 [' ' ' ' ' ' 'C' ' ' ' ' ' ' ' ' ' ' 'A']
 [' ' ' ' ' ' 'C' ' ' ' ' ' ' ' ' ' ' 'A']
 [' ' ' ' ' ' 'C' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['Y' 'Y' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '0']
 [' ' ' ' ' ' ' ' ' ' ' ' 'B' 'B' 'B' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']]
Estas son las posiciones donde están los barcos ordenador [(1, 9), (2, 9), (3, 9), (4, 9), (5, 9), 

ValueError: invalid literal for int() with base 10: ''

In [ ]:
1
